In [72]:
import numpy as np 
from numpy.linalg import inv
from scipy.optimize import minimize

In [2]:
def sortkeypicker(keynames):
    negate = set()
    for i, k in enumerate(keynames):
        if k[:1] == '-':
            keynames[i] = k[1:]
            negate.add(k[1:])
    def getit(adict):
       composite = [adict[k] for k in keynames]
       for i, (k, v) in enumerate(zip(keynames, composite)):
           if k in negate:
               composite[i] = -v
       return composite
    return getit

In [3]:
def paralel_check(v1,v2, trash_hold=0.00001):


    
    t_prod = np.cross(v1,v2)
    ar_2 = np.linalg.norm(t_prod)**2

    if ar_2<trash_hold:
        return True
    else:
        return False
    
# paralel_check(c1[0],c1[0], trash_hold=0.00001)



# v1_1= ca1+db1 => c= (v1_1-db1)/a1
# v1_2= ca2+db2 => v1_2= v1_1*a2/a1-db1*a2/a1+db2
# v1_2-v1_1*a2/a1=d(-b1*a2/a1+b2)
# b = (v1_2-v1_1*a2/a1)/(-b1*a2/a1+b2)
def decompose_to_basis(v, a,b):
    
    d = (v[1]-v[0]*a[1]/a[0])/(b[1]-b[0]*a[1]/a[0])
    c = (v[0]-d*b[0])/a[0]
    
#     print(v-c*a-d*b)
    return [c,d]

# decompose_to_basis(v=np.array([9,10]), a=np.array([2,3]),b=np.array([5,1]))

In [4]:
def point_in_cell(c, point):
    
    
    distance_to_close = 0
    s = decompose_to_basis(point, a=c[:,0],b=c[:,1])
    s1 = s[0]
    s2 = s[1]
    
    c_p = round(s1)*c[:,0]+round(s2)*c[:,1]
    distance_to_close = np.sqrt((c_p[0]-point[0])**2+(c_p[1]-point[1])**2)
    
#     print("s1:{}={}".format(round(s1),s1))
#     print("s2:{}={}".format(round(s2),s2))
#     print("distance_to_close:", distance_to_close)
    return distance_to_close

In [5]:
def get_match(cel1, cel2, exploring_range=10, tolerance=0.3, paralel_limit=0.00001):
    c = cel1
    cel1 = cel2
    cel2 = c
    c1 = (cel1[:,0],cel1[:,1])
    c2 = (cel2[:,0],cel2[:,1])
    
    candidates = []  #TODO: it can be done without list but for the beggining I preffer to keep it 
                     # in order to get a better view of the data
        
    for i in range(-exploring_range, exploring_range):
        for j in range(-exploring_range, exploring_range):
            point2 = i*c2[0]+j*c2[1]
            
            in_c1 = point_in_cell(cel1, point2)
            
            if in_c1<= tolerance:
                point_distance = point2[0]**2+point2[1]**2
                if point_distance!= 0:
                    candidates.append({"point2":point2, "in_c1":in_c1,"point_distance":point_distance}) 
                  
                    
    
    candidates = sorted(candidates, key=sortkeypicker(['point_distance', 'in_c1']), )
    
    
        
#     for c in candidates:
#         print(c)
        
    solution = [candidates[0], ]
    k=1
    while len(solution)<2 and len(candidates)>=k+2:
        k+=1
        close = candidates[k]
        min_v = close["point2"]
        if paralel_check(solution[0]["point2"], min_v,trash_hold=paralel_limit) ==False:
            solution.append(close)
        
        
    return solution
    

In [6]:
def extract_tB_strain(tB_strain, cel2):
    # Need more work
    super_cell_ = np.dot(tB_strain,cel2).transpose()
    d1 = decompose_to_basis(super_cell_[0], a= cel2[:,0],b=cel2[:,1])
    d2 = decompose_to_basis(super_cell_[1], a= cel2[:,0],b=cel2[:,1])
    print(d1)
    d1_r = np.array([round(d1[0]),round(d1[1])])
    d2_r = np.array([round(d2[0]),round(d2[1])])
    almost_super = np.array([d1_r,d2_r]).transpose()
    tB = np.dot(almost_super ,np.linalg.inv(cel2))
    strain = np.dot(tB_strain,np.linalg.inv(tB))
    
    return tB, strain

In [7]:
# diagonal strain 
def strain_tune(x, target_matrix, optimize=True):
    """
    builds a diagonal matrix with the X parameters that will have the  job of strain
    :param x:
    :param target_matrix:
    :param optimize:
    :return:
    """
    strain = np.array([[1 + x[0], 0],
                       [0, 1 + x[1]]])

    # Strain*tB_strained =tB_ => tB_strained=inv(Strain)*tB
    if optimize:
        cell_strain = np.dot(np.linalg.inv(strain), target_matrix)
        cons = 99999
        round_count = 0
        for row in cell_strain:
            for e in row:
                round_count += (round(e) - e) * (round(e) - e)  # e*e
        round_count = round_count * cons + (x[1] ** 2 + x[0] ** 2) * cons
        return round_count

    return strain

def strained_proces(target_matrix, strain_boundary):
    """
    Return a strain matrix with the propriety that multiplying  the inverse with the target matrix
    will get a matrix of integers.

    :param target_matrix:
    :param strain_boundary:
    :return:
    """

    def strain_cost(x):
        return strain_tune(x, target_matrix=target_matrix)

    x0 = np.array([0, 0])
    res = minimize(strain_cost, x0,
                   # constraints =cons,
                   bounds=strain_boundary,
                   method='L-BFGS-B')

    strain = strain_tune(res.x, target_matrix, optimize=False)
    return strain

def extract_tB_strain_diagonal(strain_tB,strain_boundary):
    print("tbs, aico",strain_tB)
    
    
    strain = strained_proces(strain_tB, strain_boundary)
    tB = np.dot(np.linalg.inv(strain),strain_tB)
    return tB,strain

In [50]:
a1 = [1,0]; a2=[0,1]
A  = np.transpose([a1,a2])

theta =  np.pi/4
R = [ [ np.cos(theta), -np.sin(theta)], [ np.sin(theta),  np.cos(theta)] ]

b1 = [3,1]#np.sqrt(2)*np.dot(R,a1)#[np.sqrt(2),0]#np.sqrt(2)*np.dot(R,a1);  #a1
b2 = a2#np.sqrt(2)*np.dot(R,a2)#[0,np.sqrt(2)]#np.sqrt(2)*np.dot(R,a2) #a2
print(b2)
#b1 = [np.sqrt(2),0]; b2=[0,np.sqrt(2)]
B  = np.transpose([b1,b2])


cel1 = np.array(A)
cel2 = np.array(B)

print("cel1:\n", cel1)
print("cel2:\n", cel2)

[0, 1]
cel1:
 [[1 0]
 [0 1]]
cel2:
 [[3 0]
 [1 1]]


In [51]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def rotation_10(v):
    th = angle_between(v, np.array([1,0]))
    rotation = np.array( [[np.cos(-th), -np.sin(-th)],
                          [np.sin(-th), np.cos(-th)]])
    return rotation

In [68]:
def brut_strain(initial_matrix):
    """
    Will generate a matrix
    that multiplied with the target matrix round it to the closest integer matrix.
    :param initial_matrix: [[float,float],[float,float]]
    :return: [[float,float],[float,float]]
    """
    round_matrix = initial_matrix.copy()
    for i in range(len(initial_matrix)):
        for j in range(len(initial_matrix[0])):
            round_matrix[i][j] = round(initial_matrix[i][j])
            
#     strain = np.dot(round_matrix, np.linalg.inv(initial_matrix))
    strain = np.dot( initial_matrix, inv(round_matrix))
    print("roud_mat:",round_matrix)
    return strain

In [69]:
a0=angle_between([cel1[0][0], cel1[1][0]],[cel2[0][0], cel2[1][0]] )
print(a0)
ro1 = rotation_10(np.array([cel1[0][0], cel1[1][0]]))
ro2 = rotation_10(np.array([cel2[0][0], cel2[1][0]]))
print("ro1",ro1)
c10= np.dot(ro1,cel1)
c20= np.dot(ro2,cel2)
print("\n'+++",c10)                  
a1 = angle_between([c10[0][0], c10[1][0]],[c20[0][0], c20[1][0]] )
print(a1)

cel1 =c10
cel2 = c20

0.0
ro1 [[ 1.  0.]
 [-0.  1.]]

'+++ [[1. 0.]
 [0. 1.]]
0.0


In [74]:
solution = get_match(cel1, cel2,exploring_range=100, tolerance=0.1, paralel_limit=0.001)
print("super_cell:\n")
for s in solution:
    print(s)
    
supper_cell = np.array([solution[0]["point2"],solution[1]["point2"]]).transpose()
tA = np.dot(supper_cell, np.linalg.inv(cel1))
strain_tB= np.dot(supper_cell,np.linalg.inv(cel2))

print("TA:\n", tA)
print("strain_TB\n",strain_tB)
# tB, strain = extract_tB_strain(strain_tB, cel2)
strain = brut_strain(strain_tB)
print("strian:", strain)
print("TB :\n",np.dot(np.linalg.inv(strain),strain_tB))

      
tB_d, strain_d= extract_tB_strain_diagonal(strain_tB,strain_boundary=[[-0.5,0.5],[-0.5,0.5]])

super_cell:

{'point2': array([-6.,  1.]), 'in_c1': 0.05198799919138583, 'point_distance': 37.0}
{'point2': array([-13.,  -1.]), 'in_c1': 0.061926003825015216, 'point_distance': 170.0}
TA:
 [[ -6. -13.]
 [  1.  -1.]]
strain_TB
 [[ -1.8973666  -13.07074766]
 [  0.31622777  -1.15950181]]
roud_mat: [[ -2. -13.]
 [  0.  -1.]]
strian: [[ 0.9486833   0.73786479]
 [-0.15811388  3.21498229]]
TB :
 [[ -2. -13.]
 [  0.  -1.]]
tbs, aico [[ -1.8973666  -13.07074766]
 [  0.31622777  -1.15950181]]


In [71]:
print("\ncel1\n",cel1)
print("cel2\n",cel2)
print("\n=====================\n")
print("\n _____\n Super cell:\n",supper_cell)
print("\n _____\n tA:\n",tA)
print("det:",np.linalg.det(tA))
print("\n _____\n strain_tB:\n",strain_tB)
print("\n _____\n tB:\n",tB)
print("\n _____\n strain:\n",strain)
print("\n=====================\n") 
print("\n _____\n diagoanl strain:\n",strain_d)
print("\n _____\n tB:\n",tB_d)
print("\n=====================\n") 


cel1
 [[1. 0.]
 [0. 1.]]
cel2
 [[ 3.16227766e+00  3.16227766e-01]
 [-4.44089210e-16  9.48683298e-01]]



 _____
 Super cell:
 [[ -6. -13.]
 [  1.  -1.]]

 _____
 tA:
 [[ -6. -13.]
 [  1.  -1.]]
det: 18.999999999999996

 _____
 strain_tB:
 [[ -1.8973666  -13.07074766]
 [  0.31622777  -1.15950181]]

 _____
 tB:
 [[ 0.         -1.        ]
 [-0.33333333  0.        ]]

 _____
 strain:
 [[ 1.01525756 -0.23300993]
 [ 0.0499307   0.2995842 ]]



 _____
 diagoanl strain:
 [[1.00417789 0.        ]
 [0.         1.11236398]]

 _____
 tB:
 [[ -1.88947258 -13.01636668]
 [  0.28428444  -1.04237627]]


